In [2]:
!pip install --upgrade "ibm-watson>=4.5.0"

  Created wheel for ibm-watson: filename=ibm_watson-4.5.0-py3-none-any.whl size=364305 sha256=401877ec13d9b0cca3f272296931b4de01d9f83dd83be3e7b93fa65f933095a1
  Stored in directory: c:\users\vedita kamat\appdata\local\pip\cache\wheels\d6\c3\2b\baf31dba99567d020d1111e4d06788d676cd5fc02e2f3b94c1
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-1.5.1-py3-none-any.whl size=44493 sha256=a78a4d4612bdc2673208ef29019f3a973e8e07fe849bce83fa2391bc5e9449fa
  Stored in directory: c:\users\vedita kamat\appdata\local\pip\cache\wheels\51\37\f5\0532ca77cb49d7405e309bb16e3bf3fd4488baabf0f326c63d
Successfully built ibm-watson ibm-cloud-sdk-core


In [3]:
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('5I3R9l0dUBGs5Ktk8FZMk_RURl-U3N-MR458L8H6htCP')
tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    authenticator=authenticator
)

tone_analyzer.set_service_url('https://api.us-south.tone-analyzer.watson.cloud.ibm.com/instances/de5f0cbc-d9cf-490d-8b1a-43a9f15b69de')

In [4]:

!pip install langdetect

  Created wheel for langdetect: filename=langdetect-1.0.8-py3-none-any.whl size=993196 sha256=9fbfaf6dfdbf8ab09923f2e7b8684f022d9c9ff28694e74ba5d692575e54bbd7
  Stored in directory: c:\users\vedita kamat\appdata\local\pip\cache\wheels\59\f6\9d\85068904dba861c0b9af74e286265a08da438748ee5ae56067
Successfully built langdetect


In [5]:
from ibm_watson import ApiException
import json
import os
import pandas as pd
from langdetect import detect
import random
import re
import time
from sklearn.model_selection import train_test_split
import ast

In [11]:
covid_data=pd.read_csv('./tweets.csv')

In [15]:
random.seed(9001)

data_api, data_test = train_test_split(data_strip_none, train_size=60, random_state=0)

# Reset index
#data_api = data_api.set_index(pd.Index(range(0,len(data_api))))
#data_test = data_test.set_index(pd.Index(range(0,len(data_test))))

# Save the dataset
data_api.to_csv('data_api.csv',index=False)
data_test.to_csv('data_test.csv',index=False)

In [13]:
data_strip_none = covid_data.set_index(pd.Index(range(0,len(covid_data))))

In [16]:
data_api = pd.read_csv('data_api.csv')

In [17]:
data_api.head()

,Unnamed: 0,text,user,location
0,1,As citizens of Mumbai grapple with a health c...,wasimkhan0730,"Mumbai, India"
1,61,INS Shardul arriving at Porbandar with 233 In...,Maheshrmurali11,"Rustaq, Oman"
2,42,Watch the video released by on Hydroxychlor...,prafull_sharma1,Raipur
3,41,Don’t hesitate to disclose your symptoms. Cal...,Myself_Om,NaN
4,4,Watch the video released by on Hydroxychlor...,Meena94916780,Shillong


In [18]:
tone_analyzer.set_disable_ssl_verification(False)

In [19]:
tones = []
index = 0
for item in data_api['text']:
    try:
        index = index + 1
        text = item
        tone_analysis = tone_analyzer.tone({'text': text}, content_type='application/json').get_result()
        tones.append(tone_analysis)
    except ApiException as ex:
        tones.append(None)
        print ("Method failed with status code " + str(ex.code) + ": " + ex.message)

In [20]:
tones_copy = tones.copy()
tones_copy = pd.DataFrame(tones_copy)

In [21]:

tones_copy.head()

,document_tone,sentences_tone
0,{'tones': []},"[{'sentence_id': 0, 'text': 'As citizens of Mu..."
1,"{'tones': [{'score': 0.733641, 'tone_id': 'ten...","[{'sentence_id': 0, 'text': 'INS Shardul arriv..."
2,{'tones': []},NaN
3,"{'tones': [{'score': 0.975663, 'tone_id': 'ten...","[{'sentence_id': 0, 'text': 'Don’t hesitate to..."
4,{'tones': []},NaN


In [22]:
data_api = pd.concat([data_api,tones_copy],axis=1)
data_api.to_csv('data_api.csv',index=False)

In [23]:
data = pd.read_csv('data_api.csv')

In [24]:
data.head()

,Unnamed: 0,text,user,location,document_tone,sentences_tone
0,1,As citizens of Mumbai grapple with a health c...,wasimkhan0730,"Mumbai, India",{'tones': []},"[{'sentence_id': 0, 'text': 'As citizens of Mu..."
1,61,INS Shardul arriving at Porbandar with 233 In...,Maheshrmurali11,"Rustaq, Oman","{'tones': [{'score': 0.733641, 'tone_id': 'ten...","[{'sentence_id': 0, 'text': 'INS Shardul arriv..."
2,42,Watch the video released by on Hydroxychlor...,prafull_sharma1,Raipur,{'tones': []},NaN
3,41,Don’t hesitate to disclose your symptoms. Cal...,Myself_Om,NaN,"{'tones': [{'score': 0.975663, 'tone_id': 'ten...","[{'sentence_id': 0, 'text': 'Don’t hesitate to..."
4,4,Watch the video released by on Hydroxychlor...,Meena94916780,Shillong,{'tones': []},NaN


In [25]:
data['document_tones'] = data.document_tone.apply(lambda x: ast.literal_eval(x)['tones'])
data.drop('document_tone',axis = 1,inplace = True)

In [26]:
for i in range(0, len(data)):
    for j in range(0, len(data.loc[i, 'document_tones'])):
        dic = data.loc[i, 'document_tones'][j]
        source = dic['tone_id']
        data.loc[i, source] = 1


,Unnamed: 0,text,user,location,sentences_tone,document_tones,tentative,analytical,joy,confident,sadness,fear
0,1,As citizens of Mumbai grapple with a health c...,wasimkhan0730,"Mumbai, India","[{'sentence_id': 0, 'text': 'As citizens of Mu...",[],NaN,NaN,NaN,NaN,NaN,NaN
1,61,INS Shardul arriving at Porbandar with 233 In...,Maheshrmurali11,"Rustaq, Oman","[{'sentence_id': 0, 'text': 'INS Shardul arriv...","[{'score': 0.733641, 'tone_id': 'tentative', '...",1.0,NaN,NaN,NaN,NaN,NaN
2,42,Watch the video released by on Hydroxychlor...,prafull_sharma1,Raipur,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN
3,41,Don’t hesitate to disclose your symptoms. Cal...,Myself_Om,NaN,"[{'sentence_id': 0, 'text': 'Don’t hesitate to...","[{'score': 0.975663, 'tone_id': 'tentative', '...",1.0,1.0,NaN,NaN,NaN,NaN
4,4,Watch the video released by on Hydroxychlor...,Meena94916780,Shillong,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
data.tail(10)

,Unnamed: 0,text,user,location,sentences_tone,document_tones,tentative,analytical,joy,confident,sadness,fear
50,87,#IndiaFightsCorona:Hydroxychloroquine (HCQ) i...,HCIGaborone,Botswana,"[{'sentence_id': 0, 'text': '#IndiaFightsCoron...",[],NaN,NaN,NaN,NaN,NaN,NaN
51,36,Not all Superheroes are Fictional! Real Life ...,Python20200383,NaN,"[{'sentence_id': 0, 'text': 'Not all Superhero...","[{'score': 0.622442, 'tone_id': 'joy', 'tone_n...",NaN,1.0,1.0,NaN,NaN,NaN
52,21,#IndiaFightsCorona:Hydroxychloroquine (HCQ) i...,govindakumar381,"3/107, Geeta Colony, New Delhi","[{'sentence_id': 0, 'text': '#IndiaFightsCoron...",[],NaN,NaN,NaN,NaN,NaN,NaN
53,83,📢 #CoronaVirusUpdates:📍 India records 0.59 de...,AirKadapa,NaN,NaN,"[{'score': 0.512022, 'tone_id': 'sadness', 'to...",NaN,NaN,NaN,NaN,1.0,NaN
54,9,"During post-#COVID19 period, startups should ...",pawanstpi,"Noida, Uttar Pradesh",NaN,"[{'score': 0.506763, 'tone_id': 'analytical', ...",NaN,1.0,NaN,NaN,NaN,NaN
55,96,#IndiaFightsCorona: #IndianRailways geared up...,NAAGPURKAR,"Mumbai, India","[{'sentence_id': 0, 'text': '#IndiaFightsCoron...",[],NaN,NaN,NaN,NaN,NaN,NaN
56,67,"For the first time, India's recovery rate ove...",AnkitBasu15,"Kolkata, India","[{'sentence_id': 0, 'text': ""For the first tim...",[],NaN,NaN,NaN,NaN,NaN,NaN
57,64,Take a look at the arrangements to be ensured...,vishyshandilya,Jodhpur,"[{'sentence_id': 0, 'text': 'Take a look at th...","[{'score': 0.952933, 'tone_id': 'analytical', ...",NaN,1.0,NaN,NaN,NaN,NaN
58,47,"Under guidance of Union Minister ji, distrib...",DwarakanathKon2,"Anantapur, India",NaN,[],NaN,NaN,NaN,NaN,NaN,NaN
59,44,"During post-#COVID19 period, startups should ...",pankaj66rawat,"Dehradun, India",NaN,"[{'score': 0.506763, 'tone_id': 'analytical', ...",NaN,1.0,NaN,NaN,NaN,NaN


In [39]:
data['analytical']=data['analytical'].fillna(0)
data['tentative']=data['tentative'].fillna(0)
data['joy']=data['joy'].fillna(0)
data['sadness']=data['sadness'].fillna(0)
data['fear']=data['fear'].fillna(0)
data['confident']=data['confident'].fillna(0)


In [40]:

data.head()

,Unnamed: 0,text,user,location,sentences_tone,document_tones,tentative,analytical,joy,confident,sadness,fear
0,1,As citizens of Mumbai grapple with a health c...,wasimkhan0730,"Mumbai, India","[{'sentence_id': 0, 'text': 'As citizens of Mu...",[],0.0,0.0,0.0,0.0,0.0,0.0
1,61,INS Shardul arriving at Porbandar with 233 In...,Maheshrmurali11,"Rustaq, Oman","[{'sentence_id': 0, 'text': 'INS Shardul arriv...","[{'score': 0.733641, 'tone_id': 'tentative', '...",1.0,0.0,0.0,0.0,0.0,0.0
2,42,Watch the video released by on Hydroxychlor...,prafull_sharma1,Raipur,NaN,[],0.0,0.0,0.0,0.0,0.0,0.0
3,41,Don’t hesitate to disclose your symptoms. Cal...,Myself_Om,NaN,"[{'sentence_id': 0, 'text': 'Don’t hesitate to...","[{'score': 0.975663, 'tone_id': 'tentative', '...",1.0,1.0,0.0,0.0,0.0,0.0
4,4,Watch the video released by on Hydroxychlor...,Meena94916780,Shillong,NaN,[],0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
cols=[1,2,3,6]
data_tentative=pd.DataFrame(data)
data_tentative=data_tentative[data_tentative.columns[cols]]
data_tentative.to_csv('data_tentative')

In [47]:
cols=[1,2,3,7]
data_analytical=pd.DataFrame(data)
data_analytical=data_analytical[data_analytical.columns[cols]]
data_analytical.to_csv('data_analytical')

In [48]:
cols=[1,2,3,8]
data_joy=pd.DataFrame(data)
data_joy=data_joy[data_joy.columns[cols]]
data_joy.to_csv('data_joy')

In [49]:
cols=[1,2,3,9]
data_confident=pd.DataFrame(data)
data_confident=data_confident[data_confident.columns[cols]]
data_confident.to_csv('data_confident')

In [50]:
cols=[1,2,3,10]
data_sadness=pd.DataFrame(data)
data_sadness=data_sadness[data_sadness.columns[cols]]
data_sadness.to_csv('data_sadness')

In [51]:
cols=[1,2,3,11]
data_fear=pd.DataFrame(data)
data_fear=data_fear[data_fear.columns[cols]]
data_fear.to_csv('data_fear')

In [52]:
data['fear'].sum()

1.0